- Medium
- 这是一种特殊的图, 2个nodes之间可以有多条directional edges

- 题目保证了从JFK出发肯定有一条走得通的path
- 那用dfs + greedy, 每次找neighbors里最小的
- 顺利的话直接就走完了
- 但也可能走到dead end, e.g. [[JFK, SFO], [SFO, JFK], [JFK, ATL], [ATL, SFO], [SFO, ATL]], path是J->A->S->A, 走不出来了
- 这种时候其实是SFO那还有另外一个cycle, 而且dfs正好可以回溯去走其他neighbors
- 那如果用一个stack来反向记录path, 那这个cycle正好可以插入到J->A->S和S->A之间
- 然后, 为了方便从小到大遍历neighbors, 那把每个点的neighbors (注意! neighbors可以存在重复值) 降序存储, 用的使用pop()即可

In [ ]:
from collections import defaultdict

class Solution:
    def findItinerary(self, tickets: List[List[str]]) -> List[str]:
        # 对每个airports, 降序存储其目的地
        # 因为tickets元素可能重复, 所以该edges也可能重复
        tickets.sort(reverse=True)
        neigs = defaultdict(list)
        for t in tickets:
            neigs[t[0]].append(t[1])
            
        # dfs访问airports
        def dfs(ap):
            while neigs[ap]:
                dfs(neigs[ap].pop())
            path.append(ap)
            
        path = []
        dfs('JFK')
        return path[::-1]

- TLE

In [33]:
import functools

class Solution:
    def findItinerary(self, tickets):
        airports = set(functools.reduce(lambda a, b: a + b, tickets))
        neigs = {a: [] for a in airports}
        for t in tickets:
            neigs[t[0]].append(t[1])
            
        # dfs
        def dfs(ap, left_trips, nxt_ava_aps, path, ans):
            if len(left_trips) == 0:
                return min(path, ans)
            for nn in nxt_ava_aps.copy():
                path += nn
                left_trips.remove([ap, nn])
                neigs[ap].remove(nn)
                ans = dfs(nn, left_trips, neigs[nn], path, ans)
                path = path[:-3]
                left_trips.append([ap, nn])
                neigs[ap].append(nn)
            return ans
        
        ans = 'Z' * (len(tickets) * 6)
        ans = dfs('JFK', tickets, neigs['JFK'], 'JFK', ans)
        return [ans[i:i+3] for i in range(0, len(ans), 3)]

In [34]:
s = Solution()
s.findItinerary([["JFK","SFO"],["JFK","ATL"],["SFO","ATL"],["ATL","JFK"],["ATL","SFO"]])

['JFK', 'ATL', 'JFK', 'SFO', 'ATL', 'SFO']